In [1]:
using MPSKit, MPSKitModels, TensorKit, TensorOperations

In [10]:
# define Hamiltionian for infinite basis with chemical potential
function BoseHubbardHamiltonian(t, U, μ, L, M)

    a⁺, a⁻ = nonsym_bosonictensors(M) # bosonic operators
    n_op = a⁺ * a⁻

    H_i = TensorMap(zeros, ComplexF64, (ℂ^(M+1))^2 ← (ℂ^(M+1))^2)
    H_i += -t * (a⁺ ⊗ a⁻ + a⁻ ⊗ a⁺) + U/2 * n_op * (n_op - id(ℂ^(M+1))) ⊗ id(ℂ^(M+1))
    H_i += -μ * n_op ⊗ id(ℂ^(M+1))

    return @mpoham sum(H_i{i,i+1} for i in (1:L))
end

BoseHubbardHamiltonian (generic function with 1 method)

In [11]:
H = BoseHubbardHamiltonian(0.1, 2, 0, 12, 5)

MPOHamiltonian{ComplexSpace, TensorMap{ComplexSpace, 2, 2, Trivial, Matrix{ComplexF64}, Nothing, Nothing}, ComplexF64}(MPSKit.SparseMPOSlice{ComplexSpace, TensorMap{ComplexSpace, 2, 2, Trivial, Matrix{ComplexF64}, Nothing, Nothing}, ComplexF64}[[TensorMap((ℂ^1 ⊗ ℂ^6) ← (ℂ^6 ⊗ ℂ^1)):
[:, :, 1, 1] =
 1.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im

[:, :, 2, 1] =
 0.0 + 0.0im  1.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im

[:, :, 3, 1] =
 0.0 + 0.0im  0.0 + 0.0im  1.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im

[:, :, 4, 1] =
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  1.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im

[:, :, 5, 1] =
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  1.0 + 0.0im  0.0 + 0.0im

[:, :, 6, 1] =
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  1.0 + 0.0im
 TensorMap((ℂ^1 ⊗ ℂ^6) ← (ℂ^6 ⊗ ℂ^3)):
[:, :, 1, 1] =
 0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im  0.0 + 0.0im

[:, :, 2, 1] =
 0.0 

In [12]:
state = FiniteMPS(rand, ComplexF64, 12, ℂ^(5+1), ℂ^20)

FiniteMPS{TensorMap{ComplexSpace, 2, 1, Trivial, Matrix{ComplexF64}, Nothing, Nothing}, TensorMap{ComplexSpace, 1, 1, Trivial, Matrix{ComplexF64}, Nothing, Nothing}}(Union{Missing, TensorMap{ComplexSpace, 2, 1, Trivial, Matrix{ComplexF64}, Nothing, Nothing}}[TensorMap((ℂ^1 ⊗ ℂ^6) ← ProductSpace(ℂ^6)):
[:, :, 1] =
 0.06878939538809115 + 0.558692314932664im  …  0.4010052925555555 + 0.09795815897077902im

[:, :, 2] =
 0.607222016937075 - 0.08597087886952025im  …  0.06346044854454284 + 0.13316617706128958im

[:, :, 3] =
 0.13821839740832914 - 0.376924479291659im  …  0.20636891990913456 - 0.06471658671257101im

[:, :, 4] =
 0.2700717777077507 - 0.06339604853569591im  …  -0.48409063630806826 + 0.567063983238377im

[:, :, 5] =
 -0.10737487332863418 - 0.16248527290111725im  …  0.043583482991300666 - 0.09297575182893146im

[:, :, 6] =
 0.15213271088292976 - 0.08838520714652959im  …  -0.2506940382638203 - 0.3629949187034887im
, TensorMap((ℂ^6 ⊗ ℂ^6) ← ProductSpace(ℂ^20)):
[:, :, 1] =
    0.16869

In [13]:
(groundstate,_) = find_groundstate(state, H, DMRG());

┌ Info: Iteraton 0 error 0.3050201982004179
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 1 error 0.002510636970519915
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 2 error 9.05838139272195e-5
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 3 error 6.197525868345965e-5
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 4 error 3.417425611762958e-5
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


...

┌ Info: Iteraton 95 error 1.3897340747109816e-9
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 96 error 1.2789796404282148e-9
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 97 error 1.1770503067200394e-9
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 98 error 1.0832420539324047e-9
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 99 error 9.96912998283084e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


In [15]:
E0 = sum(expectation_value(groundstate, H))

-0.9239709943340376 - 1.036401732103045e-16im